In [15]:
# Test file loading

import pandas as pd
import os
from cmfa.fluxomics_data.reaction import Reaction
from cmfa.fluxomics_data.tracer import Tracer, TracerExperiment

path = "../data/test_data"


In [18]:
file = "./tracers.csv"
tracer = pd.read_csv(f"{path}/{file}")

tracer


,experiment_id,met_id,tracer_id,atom_ids,ratio,atom_mdv,enrichment
0,exp1,A,[2-13C]A,[2],1.0,"[0,1]",1
1,exp2,A,"[1,2-13C]A","[1,2]",0.5,"[0.05,0.95]",1


In [19]:
file = "./reactions.csv"
reactions = pd.read_csv(f"{path}/{file}")

reactions

,model,rxn_id,rxn_eqn
0,simple_model,R1,A (abc) -> B (abc)
1,simple_model,R2,B (abc) <-> D (abc)
2,simple_model,R3,B (abc) -> C (bc) + E (a)
3,simple_model,R4,B (abc) + C (de) -> D (bcd) + E (a) + E (e)
4,simple_model,R5,D (abc) -> F (abc)


In [20]:
file = "./flux_measurements.csv"
flux = pd.read_csv(f"{path}/{file}")

flux

,experiment_id,rxn_id,flux,flux_std_error
0,exp1,R1,10.0,0.00001


In [21]:
file = "./ms_measurements.csv"
ms = pd.read_csv(f"{path}/{file}")

ms

,experiment_id,met_id,ms_id,measurement_replicate,labelled_atom_ids,unlabelled_atoms,mass_isotope,intensity,intensity_std_error,time
0,exp1,F,F1,1,"[1,2,3]",NaN,0,0.0001,0.000002,0
1,exp1,F,F1,1,"[1,2,3]",NaN,1,0.8008,0.016016,0
2,exp1,F,F1,1,"[1,2,3]",NaN,2,0.1983,0.003966,0
3,exp1,F,F1,1,"[1,2,3]",NaN,3,0.0009,0.000018,0
4,exp2,F,F1,1,"[1,2,3]",NaN,0,0.0002,0.000002,0
5,exp2,F,F1,1,"[1,2,3]",NaN,1,0.7008,0.016016,0
6,exp2,F,F1,1,"[1,2,3]",NaN,2,0.1800,0.003966,0
7,exp2,F,F1,1,"[1,2,3]",NaN,3,0.0009,0.000018,0


In [10]:
p = './../data/test_data/flux.csv'
o = './../data/test_data/flux2.csv'
f=open(p, mode='r', encoding='utf-8-sig').read()
print(f)
open('./../data/test_data/flux_measurements.csv', mode='w', encoding='utf-8').write(f)

experiment_id,rxn_id,flux,flux_std_error
exp1,R1,10.0,0.00001
exp2,R2,1.0,0.002


79

In [11]:
def convert_utf8_sig_to_utf8(input_file_path: str, output_file_path: str):
    """
    Convert a file from UTF-8-SIG encoding to UTF-8 encoding.

    Parameters
    ----------
    input_file_path : str
        The path to the input file with UTF-8-SIG encoding.
    output_file_path : str
        The path where the output file with UTF-8 encoding will be saved.
    """
    with open(input_file_path, 'r', encoding='utf-8-sig') as file:
        content = file.read()

    with open(output_file_path, 'w', encoding='utf-8') as file:
        file.write(content)

# Usage
convert_utf8_sig_to_utf8(p, o)


In [41]:
s = "2 B (de) + A (abc) -> C (bcd) + E (a) + 2E(e) + F (d)"

reactants, products = s.replace(" ", "").split('->')
print(reactants, products)

2B(de)+A(abc) C(bcd)+E(a)+2E(e)+F(d)


In [43]:
import re 
compounds = {}
atom_transitions = {}

def parse_side(side: str, sign: int):
    pattern = r"(\d*)\s*([A-Za-z]+)\s*\(([^)]+)\)"

    for match in re.finditer(pattern, side):
        coeff_str, compound, atom_transition = match.groups()
        print(match.groups())
        coeff = float(coeff_str) if coeff_str else 1.0
        coeff *= sign

        if compound not in compounds:
            compounds[compound] = 0
            atom_transitions[compound] = []

        compounds[compound] += coeff
        atom_transitions[compound].append(atom_transition)
    return compounds, atom_transitions

# Parse reactants and products
print(parse_side(reactants, -1))
print(parse_side(products, 1))

({'B': -2.0, 'A': -1.0}, {'B': ['de'], 'A': ['abc']})
({'B': -2.0, 'A': -1.0, 'C': 1.0, 'E': 3.0, 'F': 1.0}, {'B': ['de'], 'A': ['abc'], 'C': ['bcd'], 'E': ['a', 'e'], 'F': ['d']})


In [40]:
from pydantic import BaseModel, Field, validator
from typing import Dict, Optional

class Reaction(BaseModel):
    id: str
    name: Optional[str] = None
    compounds: Dict[str, float]
    atom_transition: Dict[str, str]

    def __repr__(self):
        return (
            f"Reaction id: {self.id}, name: {self.name}, "
            f"number of compounds: {len(self.compounds)}, direction: {self.direction}"
        )

    @validator('atom_transition')
    def check_atom_balance(cls, atom_transition, values):
        """
        Check if the atoms are balanced in the reaction based on atom transitions.

        Parameters
        ----------
        atom_transition : Dict[str, str]
            A dictionary mapping each compound to its atom transition pattern.

        Returns
        -------
        Dict[str, str]
            The validated atom transition dictionary.

        Raises
        ------
        ValueError
            If the atoms are not balanced.
        """
        if 'compounds' not in values:
            return atom_transition  # Cannot validate without compounds

        lhs_atoms, rhs_atoms = '', ''
        for compound, coeff in values['compounds'].items():
            transition = atom_transition.get(compound, '')
            if coeff < 0:  # Reactant
                lhs_atoms += transition * abs(int(coeff))
            else:  # Product
                rhs_atoms += transition * abs(int(coeff))

        if sorted(lhs_atoms) != sorted(rhs_atoms):
            raise ValueError(f"Unbalanced atoms in reaction {values.get('id', 'unknown')}: {lhs_atoms} != {rhs_atoms}")
        
        return atom_transition

# Example usage:
reaction = Reaction(
    id="R1",
    compounds={"A": -1, "B": -2, "C": 1, "D": 1, "E":3},
    atom_transition={"A": "abc", "B": "d", "C": "a", "D": "bc", 'E':'d'}
)


/var/folders/wy/8d18n83n56j78d4j6zkz449m0000gn/T/ipykernel_1514/4229338114.py:16: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.4/migration/
  @validator('atom_transition')


ValidationError: 1 validation error for Reaction
atom_transition
  Value error, Unbalanced atoms in reaction R1: abcdd != abcddd [type=value_error, input_value={'A': 'abc', 'B': 'd', 'C...a', 'D': 'bc', 'E': 'd'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.4/v/value_error